In [1]:
# !pip install numpy
# !pip install matplotlib
# !pip install ipywidgets
# !pip install plotly
# !pip install IPython

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
import plotly.graph_objects as go

In [3]:
import numpy as np

In [4]:
import os
OUTPUT_DIR = os.path.join('images', 'SIR')

In [5]:
from IPython.display import HTML

HTML('''
<style>
div.output_stderr {
display: none !important;
}
</style>

<script>
code_show=true;
function code_toggle() {
if (code_show){
$('div.input').hide();
} else {
$('div.input').show();
}
code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()""><input type="submit" value="Click here to toggle on/off the raw code.""></form>''')


___
___
___
.

DRIVE da DISCIPLINA:  https://drive.google.com/drive/folders/1W8Oo6xm98wCy15RpBmtPQAr3necRIWLi?usp=sharing

.

___
___
___

Links de referência:

* Wiki: https://pt.wikipedia.org/wiki/Modelo_epid%C3%AAmico
* Artigo: https://periodicos.unb.br/index.php/physicae/index  (Sistemas Complexos aplicado a modelos
epidemiológicos) --> Teor mais teórico e matemático

* Vídeo youtube (onde retirei os dados para o plot e tbm serviu de exemplo para alguns gráficos): https://www.youtube.com/watch?v=xx0FVN8MAu0

___

* #### Código Colab do vídeo do youtube (tem alguns gráficos interativos aqui do modelo SIR): https://colab.research.google.com/github/hallpaz/numerical-analysis/blob/master/SIR%20Model.ipynb#scrollTo=nhzlg8F2fIjR
___
___
___

## Modelo S.I.R


<img src='Modelo_SIR_wiki.png'>

<img src='Modelo_SIR_fluxograma.png'>


<img src='Modelo_SIR_slides.png'>

.

<img src='Modelo_SIR_hipoteses.png'>

___
___
___

In [6]:
def SIR(I0=0.0001,dt = 1,days = 20,beta=2.5,gamma=0.8,corrector=False):

    n_samples = int(days/dt) + 1
    t = np.linspace(0, days, n_samples)

    S,I,R = np.zeros(n_samples),np.zeros(n_samples),np.zeros(n_samples)
    
    # Condições Iniciais 
    R[0] = 0; I[0] = I0; S[0] = 1 - I0

    # Step equations forward in time
    for n in range(len(t)-1):
        # predictor
        S[n+1] = S[n] - dt*beta*S[n]*I[n]
        I[n+1] = I[n] + dt*beta*S[n]*I[n] - dt*gamma*I[n] 
        R[n+1] = R[n] + dt*gamma*I[n]
        
        if corrector:
            S[n+1] = S[n] - dt*beta*S[n+1]*I[n+1]
            I[n+1] = I[n] + dt*beta*S[n+1]*I[n+1] - dt*gamma*I[n+1] 
            R[n+1] = R[n] + dt*gamma*I[n+1]
    # R0 = beta*S[0]/gamma
    # print('R0', R0)
    return S, I, R

## Modelo S.I.R  


* ####  Usando np.zeros() e indexação de vetores

In [7]:
@interact_manual(I0=(0.0, 1.0, 0.01), dt=(0.1, 2))
def draw_SIR_curves(I0 = 0.11,dt = 0.1,days = 20,beta = 2.5,gamma = 0.8,corrector = False):
    
    # Chama Função SIR
    S, I, R = SIR(I0, dt, days, beta, gamma, corrector)
    
    # Cria lista do tempo corrigido
    t = [dt*i for i in range(len(S))]
    
    # Plot usando Matplotlib
    fig = plt.figure(figsize=(12,8))
    l1, l2, l3 = plt.plot(t, S, t, I, t, R)
    fig.legend((l1, l2, l3), ('S', 'I', 'R'), 'center right')
    plt.xlabel('Time'); plt.ylabel('% population'); plt.show()

interactive(children=(FloatSlider(value=0.11, description='I0', max=1.0, step=0.01), FloatSlider(value=0.1, de…

## Modelo S.I.R  


* ####  Usando listas vazias e método append --> ao invés de np.zeros() e indexação de vetores

In [8]:
def SIR2(I_0=0.0001,dt = 1,days = 20,beta=2.5,gamma=0.8,corrector=False):

    n_samples = int(days/dt) + 1
    t = np.linspace(0, days, n_samples)
    
    S_vet,I_vet,R_vet = [],[],[]
    
    # Condições Iniciais 
    R0 = 0; I0 = I_0; S0 = 1 - I_0
    

    # Step equations forward in time
    for n in range(len(t)-1):
        # predictor
        S = S0 - dt*beta*S0*I0
        I = I0 + dt*beta*S0*I0 - dt*gamma*I0 
        R = R0 + dt*gamma*I0
        S_vet.append(S); I_vet.append(I); R_vet.append(R)
        S0 = S; I0 = I; R0 = R
        
        if corrector:
            S = S0 - dt*beta*S*I
            I = I0 + dt*beta*S*I - dt*gamma*I 
            R = R0 + dt*gamma*I
            S_vet.append(S); I_vet.append(I); R_vet.append(R)
            S0 = S; I0 = I; R0 = R
    # R0 = beta*S[0]/gamma
    # print('R0', R0)
    return S_vet, I_vet, R_vet

In [9]:
@interact_manual(I_0=(0.0, 1.0, 0.01), dt=(0.1, 2))
def draw_SIR_curves(I_0 = 0.11,dt = 0.1,days = 20,beta = 2.5,gamma = 0.8,corrector = False):
    
    # Chama Função SIR
    S, I, R = SIR2(I_0, dt, days, beta, gamma, corrector)
    
    # Cria lista do tempo corrigido
    t = [dt*i for i in range(len(S))]
    
    # Plot usando Matplotlib
    fig = plt.figure(figsize=(12,8))
    l1, l2, l3 = plt.plot(t, S, t, I, t, R)
    fig.legend((l1, l2, l3), ('S', 'I', 'R'), 'center right')
    plt.xlabel('Time'); plt.ylabel('% population'); plt.show()

interactive(children=(FloatSlider(value=0.11, description='I_0', max=1.0, step=0.01), FloatSlider(value=0.1, d…

### Modelo S.I.R

* #### Usando Normalização e parâmetros externos "N" -> População Total e "np" -> nº de passos

In [18]:
# População total
N     = widgets.IntSlider(  value=10  , min=0  , max=20000, step=100, orientation='vertical',description='N')
# Número de passos
np    = widgets.IntSlider(  value=200 , min=0  , max=100000, step=100   , orientation='vertical',description='np')
# Tempo que a pessoa permanece infectada
tau    = widgets.IntSlider( value=1   , min=1  , max=100, step=1   , orientation='vertical',description='tau')

# Intervalo de tempo
dt    = widgets.FloatSlider(value=0.10, min=0.0, max=1.0, step=0.01, orientation='vertical',description='dt')
# Transmissibilidade
beta  = widgets.FloatSlider(value=2.50, min=0.0, max=5.0, step=0.01, orientation='vertical',description='beta')
# Taxa de recuperação ( g = 1/tau )
gamma = widgets.FloatSlider(value=0.80, min=0.0, max=3.0, step=0.01, orientation='vertical',description='gamma')


def SIR_Sebastián(N,np,tau, dt,beta,gamma):
    g = gamma/tau
    b = beta

    #Valores iniciais
    i=[1/N]; s=[1-i[0]]; r=[0]; t=[0]
    T=[s[0]+i[0]+r[0]]  #Total

    for it  in range(np):
      s.append(s[it]-dt*b*s[it]*i[it])
      i.append(i[it]+dt*(b*s[it]*i[it]-g*i[it]))
      r.append(1-(i[it+1]+s[it+1]))
      t.append(dt+it*dt)
      T.append(s[it+1]+i[it+1]+r[it+1])

    # Plot usando Matplotlib
    fig = plt.figure(figsize=(12,8))
    l1, l2, l3,l4 = plt.plot(t, s, t, i, t, r, t, T)
    fig.legend((l1,l2,l3,l4), ('S: Sucetíveis','I: Infectados','R: Recuperados','T: Total'), 'right')
    plt.xlabel('Time'); plt.ylabel('% population'); plt.show()
    
out2 = widgets.interactive_output(SIR_Sebastián, {'N': N,'np': np, 'tau': tau,'dt': dt, 'beta': beta,'gamma': gamma})
widgets.VBox([widgets.HBox([N, np, tau, dt, beta, gamma]), out2])

## Espaço de fases do modelo SIR    --> usando pacote GO do plotly

In [11]:
# Bibliotecas para plotar, nesse caso, usando pacote GO do plotly

import plotly.graph_objects as go
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [12]:
def draw_SI_relation(samples, dt, days, beta, gamma):
    s_list = []
    i_list = []
    corrector= 'False'
    for i0 in np.linspace(0, 1.0, samples):
        S, I, R = SIR(i0, dt, days, beta, gamma, corrector)
        s_list.append(S)
        i_list.append(I)

    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=[0.0, 1.0], y=[1.0, 0.0], mode='lines', line=dict(color='gray')))

    for k in range(len(s_list)):
        if k%2 == 0:
            fig.add_trace(go.Scatter(x=s_list[k], y=i_list[k], mode='lines',
                                     line=dict(color='blue')))
        else:
            fig.add_trace(go.Scatter(x=s_list[k], y=i_list[k], mode='lines',
                                     line=dict(color='dodgerblue', dash='dash')))
            
    for i0 in np.linspace(0.0, 0.1, int(samples / 2)):
        S, I, R = SIR(i0, dt, days, beta, gamma, corrector)
        if (i0*100)%2 == 0:
            fig.add_trace(go.Scatter(x=S, y=I, mode='lines',
                                     line=dict(color='red')))
        else:
            fig.add_trace(go.Scatter(x=S, y=I, mode='lines',
                                     line=dict(color='firebrick', dash='dash')))
            
    fig.add_trace(go.Scatter(x=[0.5]*51, y=np.linspace(0, 0.5, 51), mode='lines', line=dict(color='black')))

    fig.update_layout(
        xaxis=dict(title='S: Suscetíveis'), yaxis=dict(title='I: Infectados'),
        showlegend=True, width=800, height=800,
    )

    return fig.show()

out = interact_manual(draw_SI_relation,
    samples = widgets.IntSlider(value=7, min=0, max=50, step=1, description='amostras'),
    dt = widgets.FloatSlider(value=0.10, min=0.0, max=1.0, step=0.01,  description='dt'),
    days = widgets.IntSlider(value=20, min=0, max=100, step=1, description='days'),
    beta = widgets.FloatSlider(value=2.50, min=0.0, max=5.0, step=0.01, description='beta'),
    gamma = widgets.FloatSlider(value=0.80, min=0.0, max=3.0, step=0.01, description='gamma')
); 

display(out)

interactive(children=(IntSlider(value=7, description='amostras', max=50), FloatSlider(value=0.1, description='…

<function __main__.draw_SI_relation(samples, dt, days, beta, gamma)>

### Diagrama de fases do modelo SIR para $ \beta =2,\gamma =1$ e diferentes valores iniciais. A reta vertical é dada por  $ S={\frac {\gamma }{\beta }} $ e corresponde ao ponto de máximo da curva $ I(S) $ para as curvas que começam à direita da reta. Note que as curvas que começam sobre a reta ou à esquerda da reta possuem número de infectados sempre decrescente.

___
___
___

### Espaço de Fases do modelo SIR --> Usando Matplotlib

In [13]:
# Definição das variáves "observer" (do tipo widgets)
samples = widgets.IntSlider(value=10,    min=0,   max=50,  step=1, orientation='vertical',description='amostras')
dt    = widgets.FloatSlider(value=0.10, min=0.0, max=1.0, step=0.01, orientation='vertical',description='dt')
days  = widgets.IntSlider(  value=20  , min=0  , max=100, step=1   , orientation='vertical',description='days')
beta  = widgets.FloatSlider(value=2.50, min=0.0, max=5.0, step=0.01, orientation='vertical',description='beta')
gamma = widgets.FloatSlider(value=0.80, min=0.0, max=3.0, step=0.01, orientation='vertical',description='gamma')
corrector = widgets.Dropdown(options = ["False", "True"], value='False', description = 'Corrector')

def draw_SI_relation(samples, dt, days, beta, gamma, corrector): 
    
    s_list = []; i_list = []
    for i0 in np.linspace(0, 1.0, samples):
        S, I, R = SIR(i0, dt, days, beta, gamma, corrector)
        s_list.append(S)
        i_list.append(I)
        
    fig = plt.figure(figsize=(10, 10))
    plt.plot([0.0, 1.0], [1.0, 0.0], 'gray')
    for k in range(len(s_list)):
        plt.plot(s_list[k], i_list[k], 'blue' if k%2 else '#0000ff80')
        
    for i0 in np.linspace(0.0, 0.1, int(samples/2)):
        S, I, R = SIR(i0, dt, days, beta, gamma, corrector)
        plt.plot(S, I, 'red' if k%2 else '#ff000080')
    
    # Cria reta
    reta_S, reta_I = [], []; 
    for i in list(range(0,51,1)):
        reta_S.append(0.5); reta_I.append(i/100)
    
    plt.plot(reta_S, reta_I, 'black')
    plt.xlabel('Susceptible'); plt.ylabel('Infected'); plt.show()

    
out = widgets.interactive_output(draw_SI_relation, {'samples': samples,'dt': dt,'days': days,'beta': beta,
                                                    'gamma': gamma, 'corrector': corrector})
widgets.VBox([widgets.HBox([samples, dt, days, beta, gamma, corrector]), out])

### Comparing Euler Method with and without Predictor-Corrector technique

In [14]:
@interact_manual(I0=(0.0, 1.0, 0.01), 
                 dt=(0.1, 2),
                 show=['all', 'S', 'I', 'R'])
def compare_methods(I0 = 0.11,
                    dt = 1,
                    days = 20,
                    beta = 2.5,
                    gamma = 0.8,
                    show='all'):
    
    fe_sir = SIR(I0, dt, days, beta, gamma, False)
    pc_sir = SIR(I0, dt, days, beta, gamma, True)
    t =  [dt*i for i in range(len(fe_sir[0]))]
    legends =['S', 'I', 'R']
    
    fig, ax = plt.subplots(1, 3, figsize=(28,8)) if show == 'all' else (plt.figure(figsize=(16,8)), None)
    if show == 'all':
        for index in range(len(fe_sir)):
            ax[index].plot(t, fe_sir[index], label='{} FE'.format(legends[index]))
            ax[index].plot(t, pc_sir[index], label='{} P-C'.format(legends[index]))
            ax[index].legend(loc= 'center right')
    else:
        index = legends.index(show)
        plt.plot(t, fe_sir[index], label='{} FE'.format(legends[index]))
        plt.plot(t, pc_sir[index], label='{} P-C'.format(legends[index]))
        fig.legend(loc= 'center right')
        
    fig.suptitle('Comparinson between methods for ODE')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.show()

interactive(children=(FloatSlider(value=0.11, description='I0', max=1.0, step=0.01), FloatSlider(value=1.0, de…

___
___
___

# Modelo S.I.R.S  - com Dinâmica Vital  ( $\mu$ )

<img src='Modelo_SIRS_comDinamicaVital.png'>

<img src='Modelo_SIRS_comTaxaVital_figura.png'>



In [15]:
def SIRS(I0,dt,days,beta,gamma,mu,f):

    n_samples = int(days/dt) + 1
    t = np.linspace(0, days, n_samples)
    
    S,I,R = np.zeros(n_samples),np.zeros(n_samples),np.zeros(n_samples)
    
    # Condições Iniciais 
    R[0] = 0; I[0] = I0; S[0] = 1 - I0

    # Step equations forward in time
    for n in range(len(t)-1):
        N = S[n] + I[n] + R[n]
        S[n+1] = S[n] - dt*beta*S[n]*I[n] + dt*mu*(N-S[n]) + dt*f*R[n]
        I[n+1] = I[n] + dt*beta*S[n]*I[n] - dt*gamma*I[n] - dt*mu*I[n]
        R[n+1] = R[n] + dt*gamma*I[n] - dt*mu*R[n] - dt*f*R[n]
        
    # R0 = beta*S[0]/gamma
    # print('R0', R0)
    return S, I, R

# Definição das variáves "observer" (do tipo widgets)
I0    = widgets.FloatSlider(value=0.11, min=0.0, max=2.0, step=0.01, orientation='vertical',description='I0')
dt    = widgets.FloatSlider(value=0.10, min=0.0, max=1.0, step=0.01, orientation='vertical',description='dt')
days  = widgets.IntSlider(  value=20  , min=0  , max=100, step=1   , orientation='vertical',description='days')
beta  = widgets.FloatSlider(value=2.50, min=0.0, max=5.0, step=0.01, orientation='vertical',description='beta')
gamma = widgets.FloatSlider(value=0.80, min=0.0, max=3.0, step=0.01, orientation='vertical',description='gamma')
mu    = widgets.FloatSlider(value=0.10, min=0.000, max=1.0, step=0.01, orientation='vertical',description='mu')
f     = widgets.FloatSlider(value=0.20, min=0.0, max=2.0, step=0.001, orientation='vertical',description='f')
    
def function(I0,dt,days,beta,gamma,mu,f):
    # Chama Função SIRS
    S, I, R = SIRS(I0,dt, days, beta, gamma, mu, f)
    
    # Cria lista do tempo corrigido
    t = [dt*i for i in range(len(S))]
    
    # Plot usando Matplotlib
    fig = plt.figure(figsize=(12,8))
    l1, l2, l3 = plt.plot(t, S, t, I, t, R)
    fig.legend((l1, l2, l3), ('S: Sucetíveis', 'I: Infectados', 'R: Recuperados'), 'center right')
    plt.xlabel('Time'); plt.ylabel('% population'); plt.show()
    
out = widgets.interactive_output(function, {'I0': I0,'dt': dt, 'days': days, 'beta': beta, 
                                            'gamma': gamma, 'mu': mu, 'f': f})
widgets.VBox([widgets.HBox([I0, dt, days, beta, gamma, mu, f]), out])

<img src='Modelo_SIRS_figura.png'>

## Modelo S.I.R.S  - sem Dinâmica Vital  ( $\mu$ )

- ##  Onde: $\;\;\; \xi == f \;\;\;$  ;   $ \;\;\; \mu == \nu$

* $ \xi == f $: Taxa de reinfecção para o modelo SIRS

___

<img src='Modelo_SIRS_semDinamicaVital.png'>



In [16]:
def SIRS(I0,dt,days,beta,gamma,mu,f):

    n_samples = int(days/dt) + 1
    t = np.linspace(0, days, n_samples)
    
    S,I,R = np.zeros(n_samples),np.zeros(n_samples),np.zeros(n_samples)
    
    # Condições Iniciais 
    R[0] = 0; I[0] = I0; S[0] = 1 - I0

    # Step equations forward in time
    for n in range(len(t)-1):
        N = S[n] + I[n] + R[n]
        S[n+1] = S[n] - dt*beta*S[n]*I[n] + dt*mu*(N-S[n]) + dt*f*R[n]
        I[n+1] = I[n] + dt*beta*S[n]*I[n] - dt*gamma*I[n] - dt*mu*I[n]
        R[n+1] = R[n] + dt*gamma*I[n] - dt*mu*R[n] - dt*f*R[n]
        
    # R0 = beta*S[0]/gamma
    # print('R0', R0)
    return S, I, R

# Definição das variáves "observer" (do tipo widgets)
I0    = widgets.FloatSlider(value=0.11, min=0.0, max=2.0, step=0.01, orientation='vertical',description='I0')
dt    = widgets.FloatSlider(value=0.10, min=0.0, max=1.0, step=0.01, orientation='vertical',description='dt')
days  = widgets.IntSlider(  value=20  , min=0  , max=100, step=1   , orientation='vertical',description='days')
beta  = widgets.FloatSlider(value=2.50, min=0.0, max=5.0, step=0.01, orientation='vertical',description='beta')
gamma = widgets.FloatSlider(value=0.80, min=0.0, max=3.0, step=0.01, orientation='vertical',description='gamma')
mu    = widgets.FloatSlider(value=0.000, min=0.000, max=0.100, step=0.001, orientation='vertical',description='mu')
f     = widgets.FloatSlider(value=0.200, min=0.0, max=2.0, step=0.001, orientation='vertical',description='f')
    
def function(I0,dt,days,beta,gamma,mu,f):
    # Chama Função SIRS
    S, I, R = SIRS(I0,dt, days, beta, gamma, mu, f)
    
    # Cria lista do tempo corrigido
    t = [dt*i for i in range(len(S))]
    
    # Plot usando Matplotlib
    fig = plt.figure(figsize=(12,8))
    l1, l2, l3 = plt.plot(t, S, t, I, t, R)
    fig.legend((l1, l2, l3), ('S: Sucetíveis', 'I: Infectados', 'R: Recuperados'), 'center right')
    plt.xlabel('Time'); plt.ylabel('% population'); plt.show()
    
out = widgets.interactive_output(function, {'I0': I0,'dt': dt, 'days': days, 'beta': beta, 
                                            'gamma': gamma, 'mu': mu, 'f': f})
widgets.VBox([widgets.HBox([I0, dt, days, beta, gamma, mu, f]), out])

___
___
___

# Modelo S.E.I.R  - com Dinâmica Vital  ( $\mu$ )

<img src='Modelo_SEIR.png'>

<img src='Modelo_SEIR_figura.png'>


In [17]:
def SEIR(I0,dt,days,beta,gamma,mu,epsilon):

    n_samples = int(days/dt) + 1
    t = np.linspace(0, days, n_samples)
    
    S,E,I,R = np.zeros(n_samples),np.zeros(n_samples),np.zeros(n_samples),np.zeros(n_samples)
    
    # Condições Iniciais 
    R[0] = 0; E[0] = 0 ; I[0] = I0; S[0] = 1 - I0

    # Step equations forward in time
    for n in range(len(t)-1):
        N = S[n] + E[n] + I[n] + R[n]
        S[n+1] = S[n] - dt*beta*S[n]*I[n] + dt*mu*(N - S[n]) 
        E[n+1] = E[n] + dt*beta*S[n]*I[n] - dt*(epsilon + mu)*E[n] 
        I[n+1] = I[n] + dt*epsilon*E[n] - dt*(gamma + mu)*I[n]
        R[n+1] = R[n] + dt*gamma*I[n] - dt*mu*R[n] 
        
    # R0 = beta*S[0]/gamma
    # print('R0', R0)
    return S, E, I, R

# Definição das variáves "observer" (do tipo widgets)
I0    = widgets.FloatSlider(value=0.11, min=0.0, max=2.0, step=0.01, orientation='vertical',description='I0')
dt    = widgets.FloatSlider(value=0.10, min=0.0, max=1.0, step=0.01, orientation='vertical',description='dt')
days  = widgets.IntSlider(  value=20  , min=0  , max=1000, step=1   , orientation='vertical',description='days')
beta  = widgets.FloatSlider(value=10.0, min=0.0, max=15.0, step=0.01, orientation='vertical',description='beta')
gamma = widgets.FloatSlider(value=0.50, min=0.0, max=3.0, step=0.01, orientation='vertical',description='gamma')
mu    = widgets.FloatSlider(value=0.05, min=0.000, max=0.100, step=0.001, orientation='vertical',description='mu')
epsilon = widgets.FloatSlider(value=0.20, min=0.0, max=2.0, step=0.001, orientation='vertical',description='epsilon')
    
def function(I0,dt,days,beta,gamma,mu,epsilon):
    # Chama Função SIRS
    S, E, I, R = SEIR(I0,dt, days, beta, gamma, mu, epsilon)
    
    # Cria lista do tempo corrigido
    t = [dt*i for i in range(len(S))]
    
    # Plot usando Matplotlib
    fig = plt.figure(figsize=(12,8))
    l1, l2, l3, l4 = plt.plot(t, S,'blue', t, E,'red', t, I,'orange', t, R,'green',)
#     l1, l2, l3, l4 = plt.plot(t, S,  t, E, t, I, t, R)
#     l1, l2, l3, l4 = plt.plot(t, S, t, E, t, I, t, R)
#     l1, l2, l3, l4 = plt.plot(t, S, t, E, t, I, t, R)
#     l1, l2, l3, l4 = plt.plot(t, S, t, E, t, I, t, R)
    fig.legend((l1, l2, l3,l4), ('S: Sucetíveis','E: Expostos', 'I: Infectados', 'R: Recuperados'), 'center right')
    plt.xlabel('Time'); plt.ylabel('% population'); plt.show()
    
out = widgets.interactive_output(function, {'I0': I0,'dt': dt, 'days': days, 'beta': beta, 
                                            'gamma': gamma, 'mu': mu, 'epsilon': epsilon})
widgets.VBox([widgets.HBox([I0, dt, days, beta, gamma, mu, epsilon]), out])